# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2023/07/18



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [12]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    #df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,info.timespan.begin.to_string(),info.timespan.end.to_string() ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
    #                         info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [13]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023071700002,
 'physical_filter': 'SDSSg_65mm~blue300lpmm_qn1',
 'obs_id': 'AT_O_20230717_000002',
 'exposure_time': 2.0,
 'dark_time': 2.23319,
 'observation_type': 'engtest',
 'observation_reason': 'engtest',
 'day_obs': 20230717,
 'seq_num': 2,
 'seq_start': 2,
 'seq_end': 2,
 'group_name': '2023-07-17T21:50:00.752',
 'group_id': 2695602007520000,
 'target_name': 'DaytimeCheckout002',
 'science_program': 'unknown',
 'tracking_ra': 191.57952947983625,
 'tracking_dec': -20.115656376782933,
 'sky_angle': 99.98339884100496,
 'azimuth': 359.07787112355203,
 'zenith_angle': 9.999063503115508,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-07-17 21:50:02.864984', scale='tai', format='iso'), end=astropy.time.Time('2023-07-17 21:50:05.099000', scale='tai', format='iso'))}

In [14]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50947,2023071400042,AT_O_20230714_000042,20230714,42,2023-07-14 23:16:43.975014,2023-07-14 23:17:13.985,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.969953
50948,2023071400043,AT_O_20230714_000043,20230714,43,2023-07-14 23:17:16.470996,2023-07-14 23:17:46.482,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.970330
50949,2023071400044,AT_O_20230714_000044,20230714,44,2023-07-14 23:17:48.967983,2023-07-14 23:18:18.979,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.970706
50950,2023071700001,AT_O_20230717_000001,20230717,1,2023-07-17 21:47:06.582013,2023-07-17 21:47:08.823,engtest,DaytimeCheckout001,empty~empty,44.877822,2.0,220.938395,4.444043,172.758539,unknown,2.460143e+06,60142.907715


In [15]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [16]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50947,2023071400042,AT_O_20230714_000042,20230714,42,2023-07-14 23:16:43.975014,2023-07-14 23:17:13.985,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.969953
50948,2023071400043,AT_O_20230714_000043,20230714,43,2023-07-14 23:17:16.470996,2023-07-14 23:17:46.482,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.970330
50949,2023071400044,AT_O_20230714_000044,20230714,44,2023-07-14 23:17:48.967983,2023-07-14 23:18:18.979,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.140960,NaN,unknown,2.460140e+06,60139.970706
50950,2023071700001,AT_O_20230717_000001,20230717,1,2023-07-17 21:47:06.582013,2023-07-17 21:47:08.823,engtest,DaytimeCheckout001,empty~empty,44.877822,2.0,220.938395,4.444043,172.758539,unknown,2.460143e+06,60142.907715


## Selection of science

In [17]:
df_science = df_exposure[df_exposure.type == 'science']

In [18]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_6346/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [19]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
50213,2023070600518,AT_O_20230706_000518,20230706,518,2023-07-07 05:03:35.550001,2023-07-07 05:04:05.794,science,Photo1800-1_026,SDSSr_65mm~empty,17.928725,30.0,270.165687,-24.956565,359.999271,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.210828
50220,2023070600519,AT_O_20230706_000519,20230706,519,2023-07-07 05:04:23.395004,2023-07-07 05:04:53.633,science,Photo1800-1_027,SDSSr_65mm~empty,18.080685,30.0,270.181970,-24.956561,359.999220,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.211382
50226,2023070600520,AT_O_20230706_000520,20230706,520,2023-07-07 05:05:07.733005,2023-07-07 05:05:37.974,science,Photo1800-1_027,SDSSg_65mm~empty,18.239400,30.0,270.182005,-24.956577,359.999098,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.211895
50232,2023070600521,AT_O_20230706_000521,20230706,521,2023-07-07 05:05:52.096997,2023-07-07 05:06:22.329,science,Photo1800-1_027,SDSSi_65mm~empty,18.394836,30.0,270.181908,-24.956528,359.999309,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.212409
50242,2023070600522,AT_O_20230706_000522,20230706,522,2023-07-07 05:06:59.844005,2023-07-07 05:07:30.083,science,Photo1600-1_007,SDSSi_65mm~empty,44.842695,30.0,240.046865,-24.045367,359.995089,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.213193
50249,2023070600523,AT_O_20230706_000523,20230706,523,2023-07-07 05:07:44.257000,2023-07-07 05:08:14.501,science,Photo1600-1_007,SDSSg_65mm~empty,45.000865,30.0,240.046903,-24.045261,359.995067,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.213707
50256,2023070600524,AT_O_20230706_000524,20230706,524,2023-07-07 05:08:28.674020,2023-07-07 05:08:58.920,science,Photo1600-1_007,SDSSr_65mm~empty,45.159036,30.0,240.046866,-24.045293,359.995067,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.214221
50303,2023070600530,AT_O_20230706_000530,20230706,530,2023-07-07 05:14:10.054988,2023-07-07 05:14:40.296,science,Photo1600-1_017,SDSSr_65mm~empty,46.491133,30.0,239.949701,-23.986042,359.994716,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.218172
50309,2023070600531,AT_O_20230706_000531,20230706,531,2023-07-07 05:14:54.406991,2023-07-07 05:15:24.637,science,Photo1600-1_017,SDSSg_65mm~empty,46.649055,30.0,239.949791,-23.985973,359.994672,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.218685
50316,2023070600532,AT_O_20230706_000532,20230706,532,2023-07-07 05:15:38.742015,2023-07-07 05:16:08.993,science,Photo1600-1_017,SDSSi_65mm~empty,46.810553,30.0,239.949814,-23.986064,359.994441,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.219198


In [20]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [21]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706


In [22]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [23]:
len(df_science_selected)

341

# List of filters

In [24]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'empty~blue300lpmm_qn1', 'SDSSr_65mm~empty',
       'SDSSg_65mm~empty', 'SDSSi_65mm~empty'], dtype=object)

In [25]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [26]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [27]:
pd.set_option('display.max_rows', None)

In [28]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230511_filt_empty-holo4_003.list


/tmp/ipykernel_6346/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [29]:
N = len(all_def_science_selected_final)

In [30]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
46387,2023051100138,AT_O_20230511_000138,20230511,138,2023-05-11 23:24:15.539010,2023-05-11 23:24:45.777,science,HD185975,empty~holo4_003,62.186492,30.0,306.040759,-87.458153,291.445698,spec,2.460076e+06,60075.975180
46393,2023051100144,AT_O_20230511_000144,20230511,144,2023-05-11 23:28:28.032994,2023-05-11 23:28:58.272,science,HD185975,empty~holo4_003,62.175969,30.0,306.053558,-87.451064,296.447515,spec,2.460076e+06,60075.978102
46404,2023051100155,AT_O_20230511_000155,20230511,155,2023-05-11 23:48:27.573983,2023-05-11 23:48:57.815,science,HD185975,empty~holo4_003,62.079084,30.0,306.046756,-87.455179,287.429328,spec,2.460076e+06,60075.991986
46409,2023051100160,AT_O_20230511_000160,20230511,160,2023-05-11 23:52:23.482983,2023-05-11 23:52:53.717,science,HD185975,empty~holo4_003,62.072166,30.0,306.183639,-87.443127,302.438475,spec,2.460076e+06,60075.994716
46412,2023051100163,AT_O_20230511_000163,20230511,163,2023-05-11 23:54:16.918015,2023-05-11 23:54:47.151,science,HD185975,empty~holo4_003,62.049380,30.0,306.063205,-87.454801,287.428582,spec,2.460076e+06,60075.996029
46416,2023051100167,AT_O_20230511_000167,20230511,167,2023-05-11 23:57:46.934980,2023-05-11 23:58:17.174,science,HD38666,empty~holo4_003,59.783857,30.0,86.486332,-32.354816,188.654313,spec,2.460076e+06,60075.998460
46421,2023051100172,AT_O_20230511_000172,20230511,172,2023-05-12 00:00:56.444997,2023-05-12 00:01:26.693,science,HD38666,empty~holo4_003,60.426731,30.0,86.472896,-32.350224,203.668479,spec,2.460077e+06,60076.000653
46424,2023051100175,AT_O_20230511_000175,20230511,175,2023-05-12 00:02:49.449010,2023-05-12 00:03:19.680,science,HD38666,empty~holo4_003,60.790347,30.0,86.487001,-32.355260,188.653501,spec,2.460077e+06,60076.001961
46428,2023051100179,AT_O_20230511_000179,20230511,179,2023-05-12 00:05:19.273987,2023-05-12 00:05:49.521,science,HD38666,empty~holo4_003,61.291996,30.0,86.485521,-32.354667,189.431016,spec,2.460077e+06,60076.003695
46433,2023051100184,AT_O_20230511_000184,20230511,184,2023-05-12 00:08:32.268998,2023-05-12 00:09:02.502,science,HD38666,empty~holo4_003,61.945216,30.0,86.472273,-32.350017,204.441473,spec,2.460077e+06,60076.005929


In [31]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [32]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [33]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [34]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [35]:
df_forsave = pd.concat(all_def_science_selected_final)

In [36]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [37]:
filename_out_csv

'holosummary_20230511.csv'

In [38]:
df_forsave.to_csv(filename_out_csv) 

In [39]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [40]:
df_forsave.to_excel(filename_out_excel) 